In [38]:
import pandas as pd
import json

In [39]:
file1_df = pd.read_json('raw_data/purchase_data.json')
file2_df = pd.read_json('raw_data/purchase_data2.json')
print (len(file1_df))
print (len(file2_df))

780
78


In [40]:
#Check to make sure columns are the same between both files
file1_df.columns,file2_df.columns

(Index(['Age', 'Gender', 'Item ID', 'Item Name', 'Price', 'SN'], dtype='object'),
 Index(['Age', 'Gender', 'Item ID', 'Item Name', 'Price', 'SN'], dtype='object'))

In [41]:
#append contents of file2 to file 1
pymoli_df = file1_df.append(file2_df)
#pymoli_df = pd.merge(file1_df,file2_df, on="", how="inner")

In [42]:
#player count
playerCount = len(pymoli_df['SN'])     #alternate way: playerCount = pymoli_df['SN'].count().mean()
print(playerCount)

858


In [43]:
#purchase analysis

#number of unique items
uniqueItemCount = len(pymoli_df['Item ID'].unique())

#average purchase price
averagePurchasePrice = pymoli_df["Price"].mean()

#total number of purchases
purchaseCount = pymoli_df['Price'].count() #pymoli_df.loc[pymoli_df['Price'] == 0.00] #search through purchases to make sure non are free/0.00

#total revenue
totalRevenue = pymoli_df['Price'].sum()
#pymoli_df['Price'].value_counts()
#summary_df = pd.DataFrame({"UniqueItems":uniqueItemCount})

#summary_df["test"] = averagePurchasePrice
price_df_summary = pd.DataFrame({'Unique Item Count':uniqueItemCount,'Average Purchase Price':averagePurchasePrice,'Purchase Count':purchaseCount,'Total Revenue':totalRevenue},index=[0])
price_df_summary.head()

,Average Purchase Price,Purchase Count,Total Revenue,Unique Item Count
0,2.930571,858,2514.43,184


In [44]:
#gender demographics
genderCount = pymoli_df['Gender'].value_counts()
percentageGender = pymoli_df['Gender'].value_counts()/len(pymoli_df)*100
genderDemo_summary_df = pd.DataFrame()
genderDemo_summary_df["Gender Count"] = genderCount
genderDemo_summary_df["Percentage Gender"] = percentageGender

In [45]:
genderDemo_summary_df.head()

,Gender Count,Percentage Gender
Male,697,81.235431
Female,149,17.365967
Other / Non-Disclosed,12,1.398601


In [71]:
#purchase analysis (gender-specific)
gender_df_price =  pymoli_df.groupby(['Gender']).mean()["Price"]
gender_df_count =  pymoli_df.groupby(['Gender']).count()["Price"]

#normalized totals = divide total purchase value by count of each gender in dataset
gender_df_normalized_total=gender_df_price/gender_df_count
gender_df_normalized_total_percentage = gender_df_normalized_total * 100

genderPurchase_summary_df = pd.DataFrame({'Average Purchase Price':gender_df_price,'Purchase Count':gender_df_count,'Normalized Totals':gender_df_normalized_total,'Normalized Totals %':gender_df_normalized_total_percentage})

In [72]:
genderPurchase_summary_df.head()

,Average Purchase Price,Normalized Totals,Normalized Totals %,Purchase Count
Gender,,,,
Female,2.847584,0.019111,1.911130,149
Male,2.944448,0.004224,0.422446,697
Other / Non-Disclosed,3.155000,0.262917,26.291667,12


In [49]:
#age demographics

bins = [0,9,14,19,24,29,34,39,100] 
bin_names =["<10", "10-14", "15-19","20-24","25-29", "30-34","35-39", "40+"]
pymoli_df["Age Demographics"]= pd.cut(pymoli_df["Age"], bins, labels=bin_names)
pymoli_df.head()

,Age,Gender,Item ID,Item Name,Price,SN,Age Demographics
0,38,Male,165,Bone Crushing Silver Skewer,3.37,Aelalis34,35-39
1,21,Male,119,"Stormbringer, Dark Blade of Ending Misery",2.32,Eolo46,20-24
2,34,Male,174,Primitive Blade,2.46,Assastnya25,30-34
3,21,Male,92,Final Critic,1.36,Pheusrical25,20-24
4,23,Male,63,Stormfury Mace,1.27,Aela59,20-24


In [64]:
ageAveragePurchasePrice = pymoli_df.groupby(["Age Demographics"]).mean()['Price']
agePurchaseCount = pymoli_df.groupby(["Age Demographics"]).count()['Age']
agePurchaseValueTotal= pymoli_df.groupby(["Age Demographics"]).sum()['Price']

#normalized totals = divide total purchase value by count of each gender in dataset
age_df_normalized_total=ageAveragePurchasePrice/agePurchaseCount
age_df_normalized_total_percentage = age_df_normalized_total * 100
age_summary_df = pd.DataFrame({'Average Purchase Price':ageAveragePurchasePrice,'Purchase Count':agePurchaseCount,'Normalized Totals':age_df_normalized_total,'Normalized Totals %':age_df_normalized_total_percentage,"Total Purchase Value": agePurchaseValueTotal})

In [73]:
age_summary_df.head()

,Average Purchase Price,Normalized Totals,Normalized Totals %,Purchase Count,Total Purchase Value
Age Demographics,,,,,
10-14,2.787105,0.073345,7.334488,38,105.91
15-19,2.894653,0.020102,2.010176,144,416.83
20-24,2.923817,0.007860,0.785972,372,1087.66
25-29,2.958507,0.022078,2.207841,134,396.44
30-34,2.973803,0.041885,4.188455,71,211.14
